In [2]:
# Importa las bibliotecas necesarias
import os
import torch
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger
import wandb
import glob
import importlib
import butterfly_data_module
importlib.reload(butterfly_data_module)
from butterfly_data_module import ButterflyDataModule
from classifier import ClassifierA, ClassifierB1, ClassifierB2
from unet_autoencoder import UNetAutoencoder
import cv2
from dotenv import load_dotenv

GPU disponible: True


In [ ]:
# Ejecuta scripts externos de entrenamiento de autoencoder
!python3 train_autoencoder.py

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: miguelsanchez712000 (miguelsanchez712000-itcr). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in ./wandb/run-20241118_201716-kvvmi6lx
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run summer-serenity-16
wandb: ⭐️ View project at https://wandb.ai/miguelsanchez712000-itcr/butterfly_autoencoder
wandb: 🚀 View run at https://wandb.ai/miguelsanchez712000-itcr/butterfly_autoencoder/runs/kvvmi6lx
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name     | Type       | Params | Mode 
------------------------------------------------
0 | encoder1 | Sequential | 1.8 K  | train
1 | encoder2 | Sequential | 73.9 K | train
2 | encoder3 | Sequential | 295 K  

In [ ]:
# Ejecuta scripts externos de entrenamiento de clasificadores
!python3 train_classifiers.py

/home/miguel/.local/lib/python3.10/site-packages/lightning_fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
GPU available: True (cuda), used: True
TPU available

In [3]:
# Cargar el autoencoder desde el checkpoint para utilizar su encoder
try:
    autoencoder = UNetAutoencoder.load_from_checkpoint('/home/miguel/Desktop/IA_Proyecto3/autoencoder.ckpt')
    encoder = autoencoder.encoder
    print("Autoencoder cargado con éxito.")
except Exception as e:
    print("Error al cargar el Autoencoder:", e)

# Encuentra el último checkpoint para un modelo dado
def find_latest_checkpoint(prefix):
    """
    Busca el checkpoint más reciente basado en el prefijo del modelo.
    """
    files = glob.glob(os.path.join(ckpt_dir, f"{prefix}-epoch=*-train_loss=*.ckpt"))
    if not files:
        raise FileNotFoundError(f"No se encontró ningún archivo que coincida con el prefijo: {prefix}")
    return max(files, key=os.path.getmtime)

# Configuración de EarlyStopping para detener el entrenamiento si no hay mejora
early_stopping = EarlyStopping(monitor='train_loss', patience=5, mode='min')

# Función genérica para entrenar y evaluar un modelo
def train_and_evaluate_model(model, data_module, model_name):
    """
    Entrena y evalúa un modelo dado con el DataModule especificado.
    """
    trainer = Trainer(
        max_epochs=25,  # Número máximo de épocas para el entrenamiento
        callbacks=[early_stopping],
        accelerator='cuda',
        devices=1,
        log_every_n_steps=5
    )
    
    # Entrenar el modelo
    trainer.fit(model, data_module)
    
    # Evaluar el modelo en el conjunto de prueba
    result = trainer.test(model, dataloaders=data_module.test_dataloader())
    return result

/home/miguel/.local/lib/python3.10/site-packages/lightning_fabric/utilities/cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


Autoencoder cargado con éxito.


In [4]:
# Entrenar y evaluar modelos con diferentes proporciones de datos etiquetados y no etiquetados
for labeled_ratio in [0.1, 0.25, 0.5, 0.75, 1]:
    unlabeled_ratio = 1 - labeled_ratio
    print(f"\nEvaluando con {labeled_ratio*100:.0f}% datos etiquetados y {unlabeled_ratio*100:.0f}% datos sin etiquetar")

    # Configurar el DataModule con proporciones ajustadas
    data_module = ButterflyDataModule(
        data_dir=data_loader_dir,
        batch_size=64,
        unlabeled_ratio=unlabeled_ratio,
        labeled_ratio=labeled_ratio
    )
    data_module.setup()

    # Entrenar y evaluar ClassifierA
    classifier_a = ClassifierA(num_classes=20)
    print(f"\nEntrenando y evaluando ClassifierA con {labeled_ratio*100:.0f}% datos etiquetados")
    result_a = train_and_evaluate_model(classifier_a, data_module, "ClassifierA")
    print("Resultados de ClassifierA:", result_a)

    # Entrenar y evaluar ClassifierB1
    classifier_b1 = ClassifierB1(encoder, num_classes=20)
    print(f"\nEntrenando y evaluando ClassifierB1 con {labeled_ratio*100:.0f}% datos etiquetados")
    result_b1 = train_and_evaluate_model(classifier_b1, data_module, "ClassifierB1")
    print("Resultados de ClassifierB1:", result_b1)

    # Entrenar y evaluar ClassifierB2
    classifier_b2 = ClassifierB2(encoder, num_classes=20)
    print(f"\nEntrenando y evaluando ClassifierB2 con {labeled_ratio*100:.0f}% datos etiquetados")
    result_b2 = train_and_evaluate_model(classifier_b2, data_module, "ClassifierB2")
    print("Resultados de ClassifierB2:", result_b2)


Evaluando con 10% datos etiquetados y 90% datos sin etiquetar


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Entrenando y evaluando ClassifierA con 10% datos etiquetados


/home/miguel/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
2024-11-19 20:44:53.352188: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-19 20:44:54.594282: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                | Params | Mode 
----------------------------------------------------------
0 | model     | Sequential          | 67.2 M | train
1 | accuracy  | MulticlassAccuracy  | 0      | train
2 | precision | MulticlassPrecision | 0      | train
3 | recall    | MulticlassRecall    | 0      | train
4 | f1    

Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.5659999847412109     │
│          test_f1          │    0.18559208512306213    │
│         test_loss         │     1.622314691543579     │
│      test_precision       │    0.2399805784225464     │
│        test_recall        │    0.16712728142738342    │
└───────────────────────────┴───────────────────────────┘

Resultados de ClassifierB2: [{'test_loss': 1.622314691543579, 'test_acc': 0.5659999847412109, 'test_precision': 0.2399805784225464, 'test_recall': 0.16712728142738342, 'test_f1': 0.18559208512306213}]
